## **Proyecto 2 - Feature Engineering**

Stefanie M. Alvarez Pérez, 20002045

In [196]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scipy.stats as stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import joblib

In [197]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [198]:
pd.pandas.set_option('display.max_columns', None) #habilitamos despliegue maximo de columnas

In [199]:
data = pd.read_csv('/content/drive/MyDrive/Product Development/Proyecto 1/hotel_bookings.csv')
print(data.shape)


(119390, 32)


In [200]:
data.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [201]:
#separamos data para entrenamiento y prueba, previene el overfitting (ajuste a la estructura del dataset completo)
X_train, X_test, y_train, y_test = train_test_split(
                        data.drop(['agent', 'company'], axis=1),
                        data['adr'],
                        test_size=0.15,
                        random_state=2021) #Random State garantiza la reproducilidad del análisis

In [202]:
X_train.shape, X_test.shape

((101481, 30), (17909, 30))

### 1. Transformación del Target a Gausssiano

In [203]:
#Mejorarlo con funciones logarítmicas
y_train = np.log(y_train)
y_test = np.log(y_test)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


### 2. Missing Values

##### 2.1 Missing Values para Variables Categóricas

In [204]:
cat_vars = [var for var in data.columns if data[var].dtype == 'O']
cat_vars = cat_vars + ['is_canceled', 'is_repeated_guest']

In [205]:
#Repetir tanto para train como para test
X_train[cat_vars] = X_train[cat_vars].astype('O')
X_test[cat_vars] = X_test[cat_vars].astype('O')

In [206]:
len(cat_vars)

14

##### 2.1.1 Detección de NaN en variables categórias

In [207]:
cat_vars_with_na = [var for var in cat_vars 
                        if X_train[var].isnull().sum() > 0]

In [208]:
cat_vars_with_na

['country']

In [209]:
X_train[cat_vars_with_na].isnull().mean().sort_values(ascending = False)

country    0.004188
dtype: float64

In [210]:
#variables tratadas con etiqueta de faltante por cantidad masiva de faltantes.
vars_with_missing_string = [var for var in cat_vars_with_na
                               if X_train[var].isnull().mean() > 0.2]

#variables tratadas con procedimiento por candiad adecuada de faltantes.
vars_freq_category = [var for var in cat_vars_with_na
                               if X_train[var].isnull().mean() <= 0.2]

In [211]:
vars_with_missing_string

[]

In [212]:
vars_freq_category

['country']

##### Aplicamos cirterio para tratar data faltante

In [213]:
#Faltantes con etiqueta missing
X_train[vars_with_missing_string] = X_train[vars_with_missing_string].fillna('Missing')
X_test[vars_with_missing_string] = X_test[vars_with_missing_string].fillna('Missing')

In [214]:
#Estamos haciendo frecuency coding con la moda (por categoría para que no se repita)
for var in vars_freq_category:
    mode = X_train[var].mode()[0]
    
    X_train[var].fillna(mode, inplace=True)
    X_test[var].fillna(mode, inplace=True)
    
    print(var, "-------", mode)

country ------- PRT


In [215]:
X_train['country'].mode()[0]

'PRT'

In [216]:
X_train[cat_vars_with_na].isnull().mean().sort_values(ascending = False)

country    0.0
dtype: float64

In [217]:
#Debe salir vacío
cat_vars_with_na = [var for var in cat_vars 
                        if X_train[var].isnull().sum() > 0]
cat_vars_with_na

[]

##### 2.2 Missing Values para Variables Numéricas

In [218]:
num_vars = [var for var in X_train.columns
               if var not in cat_vars and var != 'SalePrice'] #Número de variables numéricas

In [219]:
len(num_vars)

16

In [220]:
##Numéricas con faltantes
num_vars_with_na = [var for var in num_vars
                       if X_train[var].isnull().sum() > 0] #Número de variables numéricas con faltantes



In [221]:
X_train[num_vars_with_na].isnull().mean() # % de faltantes

children    0.000039
dtype: float64

##### Aplicamos cirterio para tratar data faltante

In [222]:
for var in num_vars_with_na:
    mean_val = X_train[var].mean()
    
    print(var, mean_val)
    
    X_train[var].fillna(mean_val, inplace=True)
    X_test[var].fillna(mean_val, inplace=True) #Estamos usando la media de train para sustituir, no la de test
    

children 0.10388560954699094


In [223]:
X_train[num_vars_with_na].isnull().mean() # Ya debería salir 0

children    0.0
dtype: float64

### 3. Variables Temporales

In [224]:
def elapsed_time(df, var, ref):
    df[var] = df[ref] - df[var]
    return df

In [225]:
year_vars = [var for var in num_vars if (('Yr' in var) or ('date' in var))]
year_vars

['arrival_date_year', 'arrival_date_week_number', 'arrival_date_day_of_month']

In [226]:
ref = 'arrival_date_year'

for var in year_vars:
    if(var is not ref):
        X_train = elapsed_time(X_train, var, ref)
        X_test = elapsed_time(X_test, var, ref)

In [227]:
#Eliminamos columna YrSold debido a que guardará el valor 0
X_train.drop(['arrival_date_year'], axis=1, inplace=True)
X_test.drop(['arrival_date_year'], axis=1, inplace=True)

### 4. Transformación de Variables Numéricas

In [228]:
#Aplicamos transformación logaritmica
# Para algunas variables da error la fórmula porque tiene valores negativos
for var in ['lead_time', 'arrival_date_week_number' , 'arrival_date_day_of_month']:
    X_train[var] = np.log(X_train[var])
    X_test[var] = np.log(X_test[var])

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [229]:
#aplicamos transoformación de Yeo-Jonhson, no en todas las variables es conveniente mejorar la gráfica
#LotArea es a la única que vale la pena hacer una tranformación
X_train['booking_changes'], param = stats.yeojohnson(X_train['booking_changes'])

In [230]:
X_test['booking_changes'] = stats.yeojohnson(X_test['booking_changes'], lmbda=param) #lmbda = Correr Yeo Johnson pero con un valor ya conocido

In [231]:
print(param)

-7.551898638985714


#### 4.1 Binarización de Variables con Sesgo fuerte

In [232]:
sesgadas = ['hotel', 'stays_in_week_nights', 'previous_bookings_not_canceled', 'booking_changes', 'days_in_waiting_list']

In [233]:
#Codificar con 0 y 1
for var in sesgadas:
    X_train[var] = np.where(X_train[var] == 0, 0, 1)
    X_test[var] = np.where(X_test[var] == 0, 0, 1)

### 5. Codificación de Variables Categóricas

In [234]:
#Hacer una codificación ordinal con los valores (un diccionario)
Diccio_ADM = {'April':1, 'August':2, 'December':3, 'February':4, 'January':5, 'July':6, 'June':7, 'March':8, 'May':9,'November':10, 'October':11, 'September':12, 'Missing':0, 'NA':0, 'NaN':0}
Diccio_ART = {'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8, 'I':9,'K':10, 'L':11, 'P':12, 'Missing':0, 'NA':0, 'NaN':0}
Diccio_MS = {'Aviation':1, 'Complementary':2, 'Corporate':3, 'Direct':4, 'Groups':5, 'Offline TA/TO':6, 'Online TA':7, 'Undefined':8, 'Missing':0, 'NA':0, 'NaN':0}
Diccio_DC = {'Corporate':1, 'Direct':2, 'GDS':3, 'TA/TO':4, 'Undefined':5, 'Missing':0, 'NA':0, 'NaN':0}
Diccio_Meal = {'BB':1, 'FB':2, 'SC':3, 'Undefined':4, 'Missing':0, 'NA':0, 'NaN':0}
Diccio_CT = {'Contract':1, 'Group':2, 'Trascient':3, 'Trascient-Party':4, 'Missing':0, 'NA':0, 'NaN':0}
Diccio_RS = {'Canceled':1, 'Check-Out':2, 'No-Show':3, 'Missing':0, 'NA':0, 'NaN':0}
Diccio_DT = {'No Deposit':1, 'Non Refund':2, 'Refundable':3, 'Missing':0, 'NA':0, 'NaN':0}
Diccio_IRG = {'0':1, '1':2, 'Missing':0, 'NA':0, 'NaN':0}
Diccio_IC = {'0':1, '1':2, 'Missing':0, 'NA':0, 'NaN':0}
Diccio_IC = {'0':1, '1':2, 'Missing':0, 'NA':0, 'NaN':0}
Diccio_Hotel = {'City Hotel':1, 'resort Hotel':2, 'Missing':0, 'NA':0, 'NaN':0}

In [235]:
X_train[var] = X_train[var].map(Diccio_ART)
X_train[var] = X_train[var].map(Diccio_ADM)
X_train[var] = X_train[var].map(Diccio_MS)
X_train[var] = X_train[var].map(Diccio_DC)
X_train[var] = X_train[var].map(Diccio_Meal)
X_train[var] = X_train[var].map(Diccio_CT)
X_train[var] = X_train[var].map(Diccio_RS)
X_train[var] = X_train[var].map(Diccio_Hotel)
X_train[var] = X_train[var].map(Diccio_DT)
X_train[var] = X_train[var].map(Diccio_IRG)
X_train[var] = X_train[var].map(Diccio_IC)

In [236]:
X_test[var] = X_test[var].map(Diccio_ART)
X_test[var] = X_test[var].map(Diccio_ADM)
X_test[var] = X_test[var].map(Diccio_MS)
X_test[var] = X_test[var].map(Diccio_DC)
X_test[var] = X_test[var].map(Diccio_Meal)
X_test[var] = X_test[var].map(Diccio_CT)
X_test[var] = X_test[var].map(Diccio_RS)
X_test[var] = X_test[var].map(Diccio_Hotel)
X_test[var] = X_test[var].map(Diccio_DT)
X_test[var] = X_test[var].map(Diccio_IRG)
X_test[var] = X_test[var].map(Diccio_IC)

### 6. Feature Scaling

In [238]:
scaler = MinMaxScaler()

#scaler.fit(X_train)

#X_train = pd.DataFrame(
    #scaler.transform(X_train),
    #columns=X_train.columns)


#X_test = pd.DataFrame(
    #scaler.transform(X_test),
    #columns=X_test.columns)

In [239]:
X_train.head()

,hotel,is_canceled,lead_time,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
84964,1,0,4.477337,March,NaN,NaN,1,1,2,0.0,0,BB,PRT,Offline TA/TO,TA/TO,0,0,0,A,D,0,No Deposit,NaN,Transient,60.00,0,1,Check-Out,2016-03-07
27111,1,0,2.564949,August,NaN,NaN,0,1,2,1.0,0,HB,PRT,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,NaN,Transient,237.90,0,2,Check-Out,2016-08-20
13958,1,0,-inf,February,NaN,NaN,0,1,1,0.0,0,BB,PRT,Corporate,Corporate,0,2,1,A,D,0,No Deposit,NaN,Transient,35.00,1,2,Check-Out,2017-02-22
48359,1,0,-inf,March,NaN,NaN,0,1,2,0.0,0,BB,PRT,Complementary,Direct,0,0,0,A,A,1,No Deposit,NaN,Transient,0.00,0,1,Check-Out,2016-03-23
105085,1,0,3.135494,January,NaN,NaN,0,1,2,0.0,0,BB,NLD,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,NaN,Transient,82.88,0,0,Check-Out,2017-01-29


In [240]:
#Guardamos dataset con data preparada para entrenamiento.

X_train.to_csv('/content/drive/MyDrive/Product Development/Proyecto 1/prep_Xtrain.csv', index=False)
X_test.to_csv('/content/drive/MyDrive/Product Development/Proyecto 1/prep_Xtest.csv', index=False)

y_train.to_csv('/content/drive/MyDrive/Product Development/Proyecto 1/prep_ytrain', index=False)
y_test.to_csv('/content/drive/MyDrive/Product Development/Proyecto 1/prep_ytest', index=False)

In [241]:
joblib.dump(scaler, '/content/drive/MyDrive/Product Development/Proyecto 1/minmax_scaler.joblib')

['/content/drive/MyDrive/Product Development/Proyecto 1/minmax_scaler.joblib']

In [242]:
np.sum(X_train[X_train == 'Unf'].sum(axis=0))

0.0